## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
display(attrition_df)
attrition_df['Department'].value_counts()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Research & Development,23,2,Medical,3,41,4,...,3,3,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,Research & Development,6,1,Medical,4,42,2,...,3,1,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,Research & Development,4,3,Life Sciences,2,87,4,...,4,2,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,Sales,2,3,Medical,4,63,2,...,3,4,0,17,3,2,9,6,0,8


Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
columns = ['Age','DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'NumCompaniesWorked']
           
# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import OneHotEncoder
OverTime_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
OverTime_encoder.fit(np.array(X_train['OverTime']).reshape(-1,1))
X_train['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_train['OverTime']).reshape(-1,1))
X_test['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_test['OverTime']).reshape(-1,1))
X_train = X_train.drop('OverTime', axis=1)
X_test = X_test.drop('OverTime', axis=1)

In [8]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train_encoded = department_encoder.transform(np.array(y_train['Department']).reshape(-1,1))
y_department_test_encoded = department_encoder.transform(np.array(y_test['Department']).reshape(-1,1))
y_department_train_encoded


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [10]:
# Create a OneHotEncoder for the Attrition column
from sklearn.preprocessing import OneHotEncoder
attrition_encoded = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoded.fit(np.array(y_train['Attrition']).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train_encoded = attrition_encoded.transform(np.array(y_train['Attrition']).reshape(-1,1))
y_attrition_test_encoded = attrition_encoded.transform(np.array(y_test['Attrition']).reshape(-1,1))
y_attrition_train_encoded

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
input_features = len(X_train_scaled[0])

# Create the input layer
input = layers.Input(shape=(input_features,))
print(input_features)

# Create at least two shared layers
shared_1 = layers.Dense(64, activation='relu')(input)
shared_2 = layers.Dense(128, activation='relu')(shared_1)

10


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer_department = layers.Dense(32, activation='relu')(shared_2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(hidden_layer_department)

In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer_attrition = layers.Dense(32, activation='relu')(shared_2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train_encoded.shape[1], activation='sigmoid', name='attrition_output')(hidden_layer_attrition)

In [14]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           704         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 128)          8320        ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 32)           4128        ['dense_1[0][0]']                
                                                                                              

In [15]:
import tensorflow as tf
tf.__version__

'2.10.1'

In [16]:
# Train the model
fit_model = model.fit(X_train_scaled,{'department_output': y_department_train_encoded, 'attrition_output': y_attrition_train_encoded},epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 1ms/step - loss: 1.4566 - department_output_loss: 0.9286 - attrition_output_loss: 0.5280 - department_output_accuracy: 0.5808 - attrition_output_accuracy: 0.8067
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: 1.2110 - department_output_loss: 0.7661 - attrition_output_loss: 0.4449 - department_output_accuracy: 0.6661 - attrition_output_accuracy: 0.8276
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 1.1632 - department_output_loss: 0.7528 - attrition_output_loss: 0.4104 - department_output_accuracy: 0.6652 - attrition_output_accuracy: 0.8312
Epoch 4/100
35/35 [==============================] - 0s 2ms/step - loss: 1.1450 - department_output_loss: 0.7529 - attrition_output_loss: 0.3922 - department_output_accuracy: 0.6670 - attrition_output_accuracy: 0.8421
Epoch 5/100
35/35 [==============================] - 0s 2ms/step - loss: 1.1237 - department_output_loss: 0.7418 - attrition_out

In [17]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {'department_output': y_department_test_encoded, 'attrition_output': y_attrition_test_encoded})

# Unpack the results
model_loss = results[0]
department_loss = results[1]
attrition_loss = results[2]
department_accuracy = results[3]
attrition_accuracy = results[4]

print(f"Model Loss: {model_loss}")
print(f"Department Loss: {department_loss}")
print(f"Attrition Loss: {attrition_loss}")
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

12/12 [==============================] - 0s 1ms/step - loss: 3.5086 - department_output_loss: 2.5769 - attrition_output_loss: 0.9317 - department_output_accuracy: 0.5299 - attrition_output_accuracy: 0.8071
Model Loss: 3.5085997581481934
Department Loss: 2.576869010925293
Attrition Loss: 0.9317302703857422
Department Accuracy: 0.529891312122345
Attrition Accuracy: 0.8070651888847351


In [18]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")


Department Accuracy: 0.529891312122345
Attrition Accuracy: 0.8070651888847351


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

1. Is accuracy the best metric to use on this data? Why or why not?

Initially, when sigmoid was used for both attrition and department predictions, the model achieved 50% accuracy for department classification and 84.2% accuracy for attrition. However, the department prediction loss was high at 3.32, indicating poor performance in distinguishing between classes. This suggested that accuracy alone was not sufficient to evaluate performance, especially for the department output, as sigmoid treats multi-class problems as independent binary tasks, which can lead to suboptimal predictions.

After switching to softmax for department prediction, the department accuracy improved slightly to 52.99%, and department loss decreased to 2.58, reflecting better performance. However, the overall loss remains relatively high at 3.51, and the attrition accuracy dropped to 80.7%, with attrition loss increasing to 0.93.

This highlights that while accuracy provides insight, it may not fully capture model performance. For attrition, metrics such as precision, recall, and F1-score are more informative, particularly in imbalanced datasets. For department classification, categorical cross-entropy and AUC are better suited to evaluate the softmax output, as they provide a clearer understanding of class separation.

2. What activation functions did you choose for your output layers, and why?

Original Setup (Sigmoid for Both):

Initially, I used the sigmoid activation function for both attrition and department outputs. Sigmoid works well for binary classification tasks like attrition, where it outputs a probability between 0 and 1. However, using sigmoid for the department (multi-class) prediction resulted in poor accuracy (50%) because sigmoid treats each class independently rather than as part of a cohesive probability distribution. This led to high department loss (3.32), limiting the model’s ability to correctly classify departments.

Updated Setup (Softmax for Department):

I switched to the softmax activation function for department prediction while keeping sigmoid for attrition. Softmax is more suitable for multi-class classification because it outputs a probability distribution over all classes, ensuring the highest probability class is selected. This adjustment improved department accuracy to 52.99% and reduced department loss to 2.58. Sigmoid remains appropriate for attrition, where binary classification is required, although the accuracy slightly decreased to 80.7% with a corresponding loss increase.

3. Can you name a few ways that this model could be improved?

For Department Prediction (Primary Focus):

Feature Engineering: The model’s slight improvement in department accuracy suggests the need for more features that reflect employee skills, project history, and performance. The current feature set may not provide enough differentiation between departments.
Class Imbalance Handling: If department classes are imbalanced, apply oversampling, undersampling, or SMOTE to ensure underrepresented classes are adequately learned.

Increase Network Depth: Add more hidden layers (128–256 neurons), along with batch normalization and dropout layers, to improve generalization and reduce overfitting. This can enhance department prediction.
Embedding Layers for Categorical Data: Use embeddings for categorical variables (such as job roles or years of experience) to improve class separation.

For Attrition Prediction:

Threshold Tuning: Since attrition is binary, tuning the sigmoid threshold (e.g., lowering from 0.5 to 0.4) may improve recall, ensuring that more at-risk employees are identified.
Regularization and Dropout: Apply L2 regularization and dropout (20–30%) to hidden layers to prevent overfitting and reduce attrition loss.
Class Imbalance Adjustment: If attrition data is skewed (with more employees staying than leaving), use oversampling or synthetic data techniques to balance the classes.

General Improvements:

Learning Rate Scheduling: Introduce learning rate schedulers like ReduceLROnPlateau to dynamically adjust the learning rate during training, improving convergence and reducing overall loss.
Cross-Validation: Implement k-fold cross-validation (k=5 or 10) to ensure the model generalizes well across various data splits.
Hyperparameter Tuning: Test different optimizers (Adam, RMSprop), batch sizes (16, 32, 64), and learning rates to optimize performance further.